In [1]:
%load_ext autoreload
%autoreload 2

import torch 

from vlstm_nogatematrices import vlstm_fw_prepare_gate_preacts, vlstm_fw_nogatematrices_nostabilization, vlstm_fwbw_nogatematrices_nostabilization
from vlstm_full import vlstm_fw_torch



# vLSTM forward backward implementation

In [2]:
DTYPE = torch.float32 
DEVICE = torch.device("cuda:0")

In [3]:
B = 1
S = 5
NH = 1
DH = 6
EPS = 0.0

In [4]:
igate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
fgate_preacts = torch.randn((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
temp_Ctilde = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_D = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_QK = torch.zeros((B, NH, S, S), dtype=DTYPE, device=DEVICE)
temp_N = torch.zeros((B, NH, S, 1), dtype=DTYPE, device=DEVICE)
temp_B = torch.zeros((B, NH, S, 1), dtype=DTYPE, device=DEVICE)

In [5]:
fgate_mat, igate_mat = vlstm_fw_prepare_gate_preacts(igate_preacts, fgate_preacts)
igate_mat.shape, fgate_mat.shape

(torch.Size([1, 1, 5, 5]), torch.Size([1, 1, 5, 5]))

In [6]:
fgate_mat

tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf],
          [-0.1129,  0.0000,    -inf,    -inf,    -inf],
          [-0.9327, -0.8197,  0.0000,    -inf,    -inf],
          [-1.6315, -1.5186, -0.6988,  0.0000,    -inf],
          [-2.9347, -2.8217, -2.0020, -1.3032,  0.0000]]]], device='cuda:0')

In [7]:
igate_mat

tensor([[[[-0.9037,    -inf,    -inf,    -inf,    -inf],
          [-0.9037, -0.0643,    -inf,    -inf,    -inf],
          [-0.9037, -0.0643, -0.8249,    -inf,    -inf],
          [-0.9037, -0.0643, -0.8249, -0.1901,    -inf],
          [-0.9037, -0.0643, -0.8249, -0.1901,  0.7249]]]], device='cuda:0')

In [8]:
qs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
ks = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs = torch.randn((B, NH, S, DH), dtype=DTYPE, device=DEVICE)
vs.shape

torch.Size([1, 1, 5, 6])

## Backward NOT stabilized without input & forget gate

### Torch Autograd

In [9]:
fgate_mat_pt = fgate_mat.clone().detach().requires_grad_(True)
igate_mat_pt = igate_mat.clone().detach().requires_grad_(True)
qs_pt = qs.clone().detach().requires_grad_(True)
ks_pt = ks.clone().detach().requires_grad_(True)
vs_pt = vs.clone().detach().requires_grad_(True)
temp_Ctilde_pt = temp_Ctilde.clone().detach().requires_grad_(True)
temp_D_pt = temp_D.clone().detach().requires_grad_(True)
temp_QK_pt = temp_QK.clone().detach().requires_grad_(True)
temp_N_pt = temp_N.clone().detach().requires_grad_(True)
temp_B_pt = temp_B.clone().detach().requires_grad_(True)

In [10]:
igate_mat_pt, fgate_mat_pt

(tensor([[[[-0.9037,    -inf,    -inf,    -inf,    -inf],
           [-0.9037, -0.0643,    -inf,    -inf,    -inf],
           [-0.9037, -0.0643, -0.8249,    -inf,    -inf],
           [-0.9037, -0.0643, -0.8249, -0.1901,    -inf],
           [-0.9037, -0.0643, -0.8249, -0.1901,  0.7249]]]], device='cuda:0',
        requires_grad=True),
 tensor([[[[ 0.0000,    -inf,    -inf,    -inf,    -inf],
           [-0.1129,  0.0000,    -inf,    -inf,    -inf],
           [-0.9327, -0.8197,  0.0000,    -inf,    -inf],
           [-1.6315, -1.5186, -0.6988,  0.0000,    -inf],
           [-2.9347, -2.8217, -2.0020, -1.3032,  0.0000]]]], device='cuda:0',
        requires_grad=True))

In [11]:
retr_val_pt = vlstm_fw_nogatematrices_nostabilization(
    qs_pt, ks_pt, vs_pt, igate_mat_pt, fgate_mat_pt, 
    temp_Ctilde_pt,
    temp_D_pt, 
    temp_QK_pt,
    temp_N_pt,
    temp_B_pt,
    eps=EPS
)
retr_val_pt.shape

torch.Size([1, 1, 5, 6])

In [12]:
retr_val_pt.sum().backward()

In [13]:
temp_Ctilde_pt.grad

tensor([[[[-1.0081, -0.6488, -1.7417, -1.3857,  2.5519],
          [-1.0081, -0.6488, -1.7417, -1.3857,  2.5519],
          [-1.0081, -0.6488, -1.7417, -1.3857,  2.5519],
          [-1.0081, -0.6488, -1.7417, -1.3857,  2.5519],
          [-1.0081, -0.6488, -1.7417, -1.3857,  2.5519]]]], device='cuda:0')

In [14]:
qs_pt.grad

tensor([[[[-0.1460,  0.0956,  0.2851,  0.3477,  0.2270, -0.2115],
          [-0.3233,  0.2069,  0.1622,  0.6614,  0.1564, -0.3598],
          [-0.3338,  0.2311, -0.1683,  0.9708, -0.2470, -0.4881],
          [ 0.3376, -0.1272,  0.3181,  0.9379, -0.3482, -0.1634],
          [-1.7451,  1.4614,  2.4083,  0.4321,  2.5609, -0.1833]]]],
       device='cuda:0')

In [15]:
ks_pt.grad

tensor([[[[-0.0582,  0.0954,  0.2943, -0.2541, -0.0171,  0.2590],
          [-0.4234,  0.2434,  0.4329, -0.0063, -0.3538, -0.0048],
          [-0.0980,  0.4882,  0.5530,  0.0179, -0.0038, -0.1576],
          [-0.7273,  0.5986,  0.2099,  0.5376, -0.2532, -0.5180],
          [-2.6261, -2.1656, -1.7155, -2.9681,  1.1905,  0.7754]]]],
       device='cuda:0')

In [16]:
vs_pt.grad

tensor([[[[-0.2710, -0.2710, -0.2710, -0.2710, -0.2710, -0.2710],
          [ 0.5357,  0.5357,  0.5357,  0.5357,  0.5357,  0.5357],
          [ 0.0364,  0.0364,  0.0364,  0.0364,  0.0364,  0.0364],
          [-0.2561, -0.2561, -0.2561, -0.2561, -0.2561, -0.2561],
          [ 0.0234,  0.0234,  0.0234,  0.0234,  0.0234,  0.0234]]]],
       device='cuda:0')

In [17]:
fgate_mat_pt.grad

tensor([[[[ 0.7299,  0.0000,  0.0000,  0.0000, -0.0000],
          [-0.1318, -0.2620, -0.0000,  0.0000, -0.0000],
          [-0.1316,  0.1404,  0.7745, -0.0000, -0.0000],
          [-0.1627, -0.2030, -0.7101,  0.7395, -0.0000],
          [-0.0306, -0.0229, -0.1279, -0.3847,  0.0597]]]], device='cuda:0')

In [18]:
igate_mat_pt.grad

tensor([[[[ 0.7299,  0.0000,  0.0000,  0.0000, -0.0000],
          [-0.1318, -0.2620, -0.0000,  0.0000, -0.0000],
          [-0.1316,  0.1404,  0.7745, -0.0000, -0.0000],
          [-0.1627, -0.2030, -0.7101,  0.7395, -0.0000],
          [-0.0306, -0.0229, -0.1279, -0.3847,  0.0597]]]], device='cuda:0')

### Own backward

In [19]:
fgate_mat_obw = fgate_mat.clone().detach().requires_grad_(True)
igate_mat_obw = igate_mat.clone().detach().requires_grad_(True)
qs_obw = qs.clone().detach().requires_grad_(True)
ks_obw = ks.clone().detach().requires_grad_(True)
vs_obw = vs.clone().detach().requires_grad_(True)
temp_Ctilde_obw = temp_Ctilde.clone().detach().requires_grad_(True)
temp_D_obw = temp_D.clone().detach().requires_grad_(True)
temp_QK_obw = temp_QK.clone().detach().requires_grad_(True)
temp_N_obw = temp_N.clone().detach().requires_grad_(True)
temp_B_obw = temp_B.clone().detach().requires_grad_(True)

In [20]:
retr_val_obw = vlstm_fwbw_nogatematrices_nostabilization(
    qs_obw, ks_obw, vs_obw, igate_mat_obw, fgate_mat_obw, 
    temp_Ctilde_obw,
    temp_D_obw, 
    temp_QK_obw,
    temp_N_obw,
    temp_B_obw,
    eps=EPS
)
retr_val_obw.shape

torch.Size([1, 1, 5, 6])

In [21]:
retr_val_obw.sum().backward()

In [22]:
temp_N_obw.grad - temp_N_pt.grad

tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.]]]], device='cuda:0')

In [23]:
temp_B_obw.grad - temp_B_pt.grad

tensor([[[[0.],
          [0.],
          [0.],
          [0.],
          [0.]]]], device='cuda:0')

In [24]:
temp_QK_obw.grad - temp_QK_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

In [25]:
temp_Ctilde_obw.grad - temp_Ctilde_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

In [26]:
qs_obw.grad-qs_pt.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [27]:
ks_obw.grad-ks_pt.grad

tensor([[[[-3.7253e-09,  7.4506e-09,  0.0000e+00, -2.9802e-08, -5.5879e-09,
           -2.9802e-08],
          [-2.9802e-08,  2.9802e-08, -5.9605e-08, -5.1223e-09,  0.0000e+00,
            4.6566e-10],
          [ 0.0000e+00, -2.9802e-08,  5.9605e-08, -1.1176e-08,  4.8894e-09,
            1.4901e-08],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00,
            5.9605e-08],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.3842e-07,  0.0000e+00,
            0.0000e+00]]]], device='cuda:0')

In [28]:
vs_obw.grad-vs_pt.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [29]:
fgate_mat_obw.grad-fgate_mat_pt.grad

tensor([[[[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]]], device='cuda:0')

### Do gradients match? 

In [30]:
qs_pt.grad - qs_obw.grad

tensor([[[[0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0.]]]], device='cuda:0')

In [31]:
atol = 1e-6
rtol = 1e-6
print(f"Forward match: {torch.allclose(retr_val_pt, retr_val_obw)}")
print(f"qs match: {torch.allclose(qs_pt.grad, qs_obw.grad, atol=atol, rtol=rtol)}")
print(f"ks match: {torch.allclose(ks_pt.grad, ks_obw.grad, atol=atol, rtol=rtol)}")
print(f"vs match: {torch.allclose(vs_pt.grad, vs_obw.grad, atol=atol, rtol=rtol)}")
print(f"fgate_mat match: {torch.allclose(fgate_mat_pt.grad, fgate_mat_obw.grad, atol=atol, rtol=rtol)}")
print(f"igate_mat match: {torch.allclose(igate_mat_pt.grad, igate_mat_obw.grad, atol=atol, rtol=rtol)}")

Forward match: True
qs match: True
ks match: True
vs match: True
fgate_mat match: True
igate_mat match: True


## Forward without input & forget gate

In [32]:
retr_vals = vlstm_fw_nogatematrices_nostabilization(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igate_mat,
    fgate_preact=fgate_mat,
    temp_Ctilde=temp_Ctilde,
    temp_D=temp_D,
    temp_QK=temp_QK,
    temp_N=temp_N,
    temp_B=temp_B,
    eps=EPS,
)
retr_vals.shape

torch.Size([1, 1, 5, 6])

In [33]:
retr_vals_fwbw = vlstm_fwbw_nogatematrices_nostabilization(
    queries=qs,
    keys=ks,
    values=vs,
    igate_preact=igate_mat,
    fgate_preact=fgate_mat,
    temp_Ctilde=temp_Ctilde,
    temp_D=temp_D,
    temp_QK=temp_QK,
    temp_N=temp_N,
    temp_B=temp_B,
    eps=EPS,
)
retr_vals_fwbw.shape

torch.Size([1, 1, 5, 6])

### Check if it equals the full version:

In [34]:
# check if equals the full version
retr_vals_full = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igate_preacts, fgate_preact=fgate_preacts)
retr_vals_full.shape

torch.Size([1, 1, 5, 6])

In [35]:
# The implementations match!!!
retr_vals - retr_vals_full

tensor([[[[-2.3842e-07, -7.4506e-08,  2.3842e-07,  1.7881e-07,  1.1921e-07,
            7.4506e-08],
          [-3.2783e-07, -4.6566e-09,  1.0431e-07,  3.8743e-07, -4.1723e-07,
           -2.9802e-08],
          [ 4.1723e-07, -5.9605e-08, -4.4703e-08, -1.7881e-07,  2.3842e-07,
            8.9407e-08],
          [-3.5763e-07,  3.8743e-07,  1.4901e-08,  5.9605e-07, -2.9802e-07,
           -7.1526e-07],
          [-6.5565e-07, -1.7881e-07, -2.4214e-08, -1.1921e-07, -7.4506e-07,
            6.2585e-07]]]], device='cuda:0')

In [36]:
retr_vals_fwbw - retr_vals_full

tensor([[[[-2.3842e-07, -7.4506e-08,  2.3842e-07,  1.7881e-07,  1.1921e-07,
            7.4506e-08],
          [-3.2783e-07, -4.6566e-09,  1.0431e-07,  3.8743e-07, -4.1723e-07,
           -2.9802e-08],
          [ 4.1723e-07, -5.9605e-08, -4.4703e-08, -1.7881e-07,  2.3842e-07,
            8.9407e-08],
          [-3.5763e-07,  3.8743e-07,  1.4901e-08,  5.9605e-07, -2.9802e-07,
           -7.1526e-07],
          [-6.5565e-07, -1.7881e-07, -2.4214e-08, -1.1921e-07, -7.4506e-07,
            6.2585e-07]]]], device='cuda:0')